In [4]:
import os
import re
import joblib
import librosa
import torch
import wave
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
from random import shuffle

In [7]:
train_path_dir = '/home/nz32/speech2text/aishell/data_aishell/wav/train'
dev_path_dir = '/home/nz32/speech2text/aishell/data_aishell/wav/dev'
#数据预处理
train_files_path = []
dev_files_path = []

# 把所有train文件的名字放在装文件的list里
def recur_train(rootdir):
    for root, dirs, files in tqdm(os.walk(rootdir)):
        for file in files:
            if 'DS_Store' in file:
                continue
            train_files_path.append(os.path.join(root,file))
        for dir in dirs:
            recur_train(dir)
            
# 把所有dev文件的名字放在目录的list里
def recur_dev(rootdir):
    for root, dirs, files in tqdm(os.walk(rootdir)):
        for file in files:
            if 'DS_Store' in file:
                continue
            dev_files_path.append(os.path.join(root,file))
        for dir in dirs:
            recur_dev(dir)
            
recur_train(train_path_dir)
recur_dev(dev_path_dir)

0it [00:00, ?it/s]
0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0

In [17]:
print('train_files_path len:', len(train_files_path))
print('dev_files_path len:', len(dev_files_path))
all_files_path = train_files_path+dev_files_path
print('all files len: ', len(all_files_path))


train_files_path len: 120418
dev_files_path len: 14331
all files len:  134749


In [15]:
# 读取transcript，处理成字典形式{'BAC009S0002W0125': '各地政府便纷纷跟进'}
_d={}
with open('/home/nz32/speech2text/aishell/data_aishell/transcript/aishell_transcript_v0.8.txt') as f:
    data = f.readlines()
    for i in tqdm(data):
        k,v = re.split('\s+', i, 1)
        _d[k.strip()] = v.replace('\n','').replace('\t','').replace(' ','')


100%|██████████| 141600/141600 [00:00<00:00, 472576.95it/s]


In [30]:
# 把wav文件与transcript对齐
res_train = []
for file in tqdm(train_files_path):
    file_name = file.split('/')[-1][:-4] # 从list末尾反向截取到导数第四个元素之间'BAC009S0016W0295'
    if file_name in _d:
        res_train.append((file, _d[file_name]))
        
res_dev =[]
for file in tqdm(dev_files_path):
    file_name = file.split('/')[-1][:-4]
    if file_name in _d:
        res_dev.append((file, _d[file_name]))
        

100%|██████████| 14331/14331 [00:00<00:00, 1052818.57it/s]


In [38]:
all_words = list(set(''.join([v for v in _d.values()]))) #构造包含全部transcript中汉字的字典
all_words = ['_'] + all_words[:27] + [' '] + all_words[27:]

In [41]:
len(all_words)

4335

In [45]:
pd.DataFrame(res_train).to_csv('/home/nz32/git/An-aid-to-learning-to-read-foreign-languages/speech2text/data_aishell/train.index', index=False, header =None)
pd.DataFrame(res_dev).to_csv('/home/nz32/git/An-aid-to-learning-to-read-foreign-languages/speech2text/data_aishell/dev.index', index=False, header =None)


In [50]:
joblib.dump(all_words, 'labels.gz')

['labels.gz']

In [51]:
with open('/home/nz32/git/An-aid-to-learning-to-read-foreign-languages/speech2text/data_aishell/train.index') as f:
    idx = f.readlines()
idx = [x.strip().split(",", 1) for x in idx]

[['/home/nz32/speech2text/aishell/data_aishell/wav/train/S0016/BAC009S0016W0295.wav',
  '苹果占有全球智能手机市场百分之十一点七的市场份额'],
 ['/home/nz32/speech2text/aishell/data_aishell/wav/train/S0016/BAC009S0016W0479.wav',
  '八十三名中企工人因公司欠薪被困蒙古挖野菜充饥'],
 ['/home/nz32/speech2text/aishell/data_aishell/wav/train/S0016/BAC009S0016W0162.wav',
  '目前还处于互联网家装的发展初期'],
 ['/home/nz32/speech2text/aishell/data_aishell/wav/train/S0016/BAC009S0016W0128.wav',
  '悦家的身份其实是多品牌多集成的综合营销管道'],
 ['/home/nz32/speech2text/aishell/data_aishell/wav/train/S0016/BAC009S0016W0359.wav',
  '汤普森以零点零三秒之差屈于亚军'],
 ['/home/nz32/speech2text/aishell/data_aishell/wav/train/S0016/BAC009S0016W0478.wav',
  '在庆平胡同的一间小平房里'],
 ['/home/nz32/speech2text/aishell/data_aishell/wav/train/S0016/BAC009S0016W0147.wav',
  '要说服精明的投资人'],
 ['/home/nz32/speech2text/aishell/data_aishell/wav/train/S0016/BAC009S0016W0277.wav',
  '或能看清自己无法进入智能手机竞争市场的现实'],
 ['/home/nz32/speech2text/aishell/data_aishell/wav/train/S0016/BAC009S0016W0321.wav',
  '就会出现制造类企业引进工业机器人代替工人的浪潮'],
 

In [56]:
# test dump result
# all_words = joblib.load('/home/nz32/git/An-aid-to-learning-to-read-foreign-languages/speech2text/data_aishell/labels.gz')
# len(res_train)
# train_pd = pd.DataFrame(res_train)
# train_pd

In [57]:
all_words

['_',
 '匣',
 '胶',
 '讶',
 '泔',
 '双',
 '棕',
 '祠',
 '摧',
 '昨',
 '掐',
 '枉',
 '童',
 '桂',
 '扶',
 '晒',
 '芯',
 '十',
 '起',
 '垛',
 '郁',
 '疆',
 '娃',
 '习',
 '必',
 '疲',
 '胆',
 '态',
 ' ',
 '渊',
 '愤',
 '荷',
 '蔫',
 '栏',
 '孝',
 '苟',
 '贞',
 '羡',
 '世',
 '克',
 '岛',
 '元',
 '茂',
 '返',
 '溪',
 '火',
 '截',
 '黄',
 '良',
 '沏',
 '叹',
 '丽',
 '湖',
 '荟',
 '钥',
 '谶',
 '撕',
 '阻',
 '泊',
 '影',
 '石',
 '祯',
 '羹',
 '梵',
 '递',
 '斡',
 '铭',
 '漕',
 '曹',
 '冥',
 '洛',
 '潇',
 '尧',
 '峨',
 '桎',
 '由',
 '莴',
 '犀',
 '髅',
 '彷',
 '嗣',
 '芪',
 '蓝',
 '局',
 '峡',
 '棒',
 '组',
 '敌',
 '讪',
 '鸭',
 '辖',
 '车',
 '樟',
 '懒',
 '嗪',
 '坍',
 '攀',
 '恶',
 '锵',
 '磋',
 '加',
 '络',
 '玩',
 '店',
 '捕',
 '质',
 '蒂',
 '句',
 '闵',
 '芦',
 '馆',
 '泠',
 '搀',
 '煞',
 '钵',
 '佐',
 '洽',
 '璧',
 '妍',
 '鄙',
 '浚',
 '参',
 '详',
 '奏',
 '艰',
 '焉',
 '惯',
 '射',
 '妊',
 '扎',
 '遛',
 '广',
 '鲶',
 '寰',
 '江',
 '宗',
 '赁',
 '蓉',
 '翠',
 '日',
 '栗',
 '裤',
 '勒',
 '酋',
 '比',
 '瞩',
 '稍',
 '厌',
 '亳',
 '娜',
 '甘',
 '嘉',
 '玫',
 '葆',
 '掠',
 '渚',
 '厂',
 '琢',
 '楚',
 '憾',
 '杞',
 '暖',
 '氧',
 '奚',
 '观',
 '颂',
 '茬'

120098

,0,1
0,/home/nz32/speech2text/aishell/data_aishell/wa...,苹果占有全球智能手机市场百分之十一点七的市场份额
1,/home/nz32/speech2text/aishell/data_aishell/wa...,八十三名中企工人因公司欠薪被困蒙古挖野菜充饥
2,/home/nz32/speech2text/aishell/data_aishell/wa...,目前还处于互联网家装的发展初期
3,/home/nz32/speech2text/aishell/data_aishell/wa...,悦家的身份其实是多品牌多集成的综合营销管道
4,/home/nz32/speech2text/aishell/data_aishell/wa...,汤普森以零点零三秒之差屈于亚军
...,...,...
120093,/home/nz32/speech2text/aishell/data_aishell/wa...,之所以在本市楼市低迷的情况下
120094,/home/nz32/speech2text/aishell/data_aishell/wa...,将地铁公司诉至西城法院索赔各项损失十七万馀元
120095,/home/nz32/speech2text/aishell/data_aishell/wa...,乘客乘坐地铁被挤骨折起诉地铁公司获赔两万余元
120096,/home/nz32/speech2text/aishell/data_aishell/wa...,移动通信产品如平板电脑等
